In [1]:
import pandas as pd
import re

# Load the CSV file
file_path = 'raw_0819.csv'
df = pd.read_csv(file_path, header=None)

# Use the first row as the header (column names)
df.columns = df.iloc[0]

# Drop the first three rows (including the now redundant header row)
df = df.drop([0, 1, 2])

# Reset the index
df = df.reset_index(drop=True)

# Define the two groups of columns for accuracy calculation
group_1_columns = ['Q39', 'Q107', 'Q111', 'Q114', 'Q117', 'Q119', 'Q123', 'Q126', 'Q129', 'Q131']
group_2_columns = ['Q180', 'Q184', 'Q188', 'Q191', 'Q194', 'Q197', 'Q200', 'Q202', 'Q206', 'Q209']

# Convert values to 1 or 0 based on the condition
df[group_1_columns] = df[group_1_columns].applymap(lambda x: 1 if x == '1' else 0)
df[group_2_columns] = df[group_2_columns].applymap(lambda x: 1 if x == '1' else 0)

# Calculate the group accuracies
df['GROUP_1_LC_ACCURACY'] = df[group_1_columns].mean(axis=1)
df['GROUP_2_LC_ACCURACY'] = df[group_2_columns].mean(axis=1)

# Define the additional columns you want to keep
additional_columns = [
    'Q1', 'Q2', 'Q3', 'Q5', 'Q6', 'Q173', 'Q214', 
    'LEVEL_OF_EMOTIONAL_AROUSAL', 'FIRST_LEVEL_OF_COGNITIVE_LOAD', 'FIRST_WORD_LIST'
]

# Combine the additional columns with the accuracy columns
final_columns = additional_columns + ['GROUP_1_LC_ACCURACY', 'GROUP_2_LC_ACCURACY']

# Keep only the required columns
df_final = df[final_columns]

# Rename the columns according to the specified conventions
rename_mapping = {
    'Q1': 'INITIALS',
    'Q2': 'GENDER',
    'Q3': 'AGE',
    'Q5': 'VISION',
    'Q6': 'HEARING',
    'Q173': 'LIST_1',
    'Q214': 'LIST_2',
    'LEVEL_OF_EMOTIONAL_AROUSAL': 'LEVEL_OF_EMOTIONAL_AROUSAL',
    'FIRST_LEVEL_OF_COGNITIVE_LOAD': 'FIRST_LEVEL_OF_COGNITIVE_LOAD',
    'FIRST_WORD_LIST': 'FIRST_WORD_LIST',
    'GROUP_1_LC_ACCURACY': 'GROUP_1_LC_ACCURACY',
    'GROUP_2_LC_ACCURACY': 'GROUP_2_LC_ACCURACY'
}

# Apply the renaming
df_final = df_final.rename(columns=rename_mapping)

# Ensure all columns are in uppercase with underscores
df_final.columns = [col.upper().replace(' ', '_') for col in df_final.columns]

# Capitalize initials and remove spaces
df_final['INITIALS'] = df_final['INITIALS'].str.replace(" ", "").str.upper()

# Normalize gender values
def normalize_gender(value):
    if value.lower().startswith('f'):
        return 'F'
    elif value.lower().startswith('m'):
        return 'M'
    else:
        return 'OTHER'

df_final['GENDER'] = df_final['GENDER'].apply(normalize_gender)

# Convert vision and hearing columns to true/false
def normalize_vision_hearing(value):
    return 'TRUE' if value in ['1', '2'] else 'FALSE'

df_final['VISION'] = df_final['VISION'].apply(normalize_vision_hearing)
df_final['HEARING'] = df_final['HEARING'].apply(normalize_vision_hearing)

# Reference word lists
list_1_reference = [
    "mountain", "speaker", "wheelchair", "glass", "house", "fork", "spouse", "juice", 
    "classroom", "shoe", "bodega", "baseball", "lumberjack", "cloud", "armadillo"]

list_2_reference = [
    "pants", "tequila", "grass", "bell", "helmet", "tortilla", "ambulance", "bicep", 
    "purple", "tornado", "recycling", "carnival", "spectate", "giraffe", "roommate"]

# Function to count unique matches with better tokenization
def count_unique_matches(recalled_list, reference_list):
    # Tokenize the recalled list (splitting on spaces, commas, and other punctuation)
    recalled_words = re.findall(r'\b\w+\b', recalled_list.lower())
    recalled_set = set(recalled_words)
    
    # Count matches with the reference list
    return len(recalled_set.intersection(reference_list))

# Calculate the word match counts
def calculate_match_counts(row):
    if row['FIRST_WORD_LIST'] == '1':
        list_1_match = count_unique_matches(row['LIST_1'], list_1_reference)
        list_2_match = count_unique_matches(row['LIST_2'], list_2_reference)
    else:
        list_1_match = count_unique_matches(row['LIST_1'], list_2_reference)
        list_2_match = count_unique_matches(row['LIST_2'], list_1_reference)
    return pd.Series([list_1_match, list_2_match])

df_final[['LIST_1_MATCH_COUNT', 'LIST_2_MATCH_COUNT']] = df_final.apply(calculate_match_counts, axis=1)

# Reorder and rename based on the first level of cognitive load
def reorder_based_on_cognitive_load(row):
    if row['FIRST_LEVEL_OF_COGNITIVE_LOAD'].lower() == 'low':
        return pd.Series({
            'LOW_COGNITIVE_LOAD_ACCURACY': row['GROUP_1_LC_ACCURACY'],
            'HIGH_COGNITIVE_LOAD_ACCURACY': row['GROUP_2_LC_ACCURACY'],
            'LOW_COGNITIVE_LOAD_MATCH_COUNT': row['LIST_1_MATCH_COUNT'],
            'HIGH_COGNITIVE_LOAD_MATCH_COUNT': row['LIST_2_MATCH_COUNT']
        })
    else:
        return pd.Series({
            'LOW_COGNITIVE_LOAD_ACCURACY': row['GROUP_2_LC_ACCURACY'],
            'HIGH_COGNITIVE_LOAD_ACCURACY': row['GROUP_1_LC_ACCURACY'],
            'LOW_COGNITIVE_LOAD_MATCH_COUNT': row['LIST_2_MATCH_COUNT'],
            'HIGH_COGNITIVE_LOAD_MATCH_COUNT': row['LIST_1_MATCH_COUNT']
        })

df_final[['LOW_COGNITIVE_LOAD_ACCURACY', 'HIGH_COGNITIVE_LOAD_ACCURACY', 'LOW_COGNITIVE_LOAD_MATCH_COUNT', 'HIGH_COGNITIVE_LOAD_MATCH_COUNT']] = df_final.apply(reorder_based_on_cognitive_load, axis=1)

# Drop the old columns after reordering
df_final = df_final.drop(columns=['GROUP_1_LC_ACCURACY', 'GROUP_2_LC_ACCURACY', 'LIST_1_MATCH_COUNT', 'LIST_2_MATCH_COUNT'])

# Prepare the first sheet (essential information)
sheet1_columns = [
    'INITIALS', 'GENDER', 'AGE', 'VISION', 'HEARING', 'LEVEL_OF_EMOTIONAL_AROUSAL',
    'LOW_COGNITIVE_LOAD_ACCURACY', 'HIGH_COGNITIVE_LOAD_ACCURACY',
    'LOW_COGNITIVE_LOAD_MATCH_COUNT', 'HIGH_COGNITIVE_LOAD_MATCH_COUNT'
]
sheet1 = df_final[sheet1_columns]

# Prepare the second sheet (detailed information)
sheet2_columns = [
    'INITIALS', 'GENDER', 'AGE', 'VISION', 'HEARING', 'LIST_1', 'LIST_2',
    'LEVEL_OF_EMOTIONAL_AROUSAL', 'FIRST_LEVEL_OF_COGNITIVE_LOAD', 'FIRST_WORD_LIST',
    'LOW_COGNITIVE_LOAD_ACCURACY', 'HIGH_COGNITIVE_LOAD_ACCURACY',
    'LOW_COGNITIVE_LOAD_MATCH_COUNT', 'HIGH_COGNITIVE_LOAD_MATCH_COUNT'
]
sheet2 = df_final[sheet2_columns]

# Write both sheets to an Excel file
with pd.ExcelWriter('preprocessed_data.xlsx') as writer:
    sheet1.to_excel(writer, sheet_name='Essential Information', index=False)
    sheet2.to_excel(writer, sheet_name='Detailed Information', index=False)

/var/folders/25/h4h_hr5535g3kclczsqww0880000gn/T/ipykernel_20672/1235108827.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[group_1_columns] = df[group_1_columns].applymap(lambda x: 1 if x == '1' else 0)
/var/folders/25/h4h_hr5535g3kclczsqww0880000gn/T/ipykernel_20672/1235108827.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[group_2_columns] = df[group_2_columns].applymap(lambda x: 1 if x == '1' else 0)


AttributeError: 'float' object has no attribute 'lower'

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import AnovaRM

# Load the existing Excel file
file_path = 'preprocessed_data.xlsx'
df_final = pd.read_excel(file_path, sheet_name='Detailed Information')

# Calculate Means
mean_stats = df_final[['LOW_COGNITIVE_LOAD_ACCURACY', 'HIGH_COGNITIVE_LOAD_ACCURACY', 
                       'LOW_COGNITIVE_LOAD_MATCH_COUNT', 'HIGH_COGNITIVE_LOAD_MATCH_COUNT']].mean()

# Prepare the data for ANOVA
# Melt the data to long format for ANOVA
df_anova = pd.melt(df_final, 
                   id_vars=['INITIALS', 'GENDER', 'AGE', 'VISION', 'HEARING', 'LEVEL_OF_EMOTIONAL_AROUSAL'], 
                   value_vars=['LOW_COGNITIVE_LOAD_ACCURACY', 'HIGH_COGNITIVE_LOAD_ACCURACY'], 
                   var_name='COGNITIVE_LOAD', 
                   value_name='ACCURACY')

# Map cognitive load to low/high for simplicity
df_anova['COGNITIVE_LOAD'] = df_anova['COGNITIVE_LOAD'].map({
    'LOW_COGNITIVE_LOAD_ACCURACY': 'Low', 
    'HIGH_COGNITIVE_LOAD_ACCURACY': 'High'
})

# Conduct 2-way mixed measures ANOVA
anova_model = AnovaRM(df_anova, 'ACCURACY', 'INITIALS', within=['COGNITIVE_LOAD'], between=['LEVEL_OF_EMOTIONAL_AROUSAL'])
anova_results = anova_model.fit()

# Collect results
anova_table = anova_results.summary().tables[0].as_html()
anova_df = pd.read_html(anova_table, header=0, index_col=0)[0]

NotImplementedError: Between subject effect not yet supported!

In [ ]:
# Prepare the output Excel file
with pd.ExcelWriter(file_path, mode='a', if_sheet_exists='new') as writer:
    # Write the mean statistics to a new sheet
    mean_stats.to_excel(writer, sheet_name='Mean Statistics', index=True)

    # Write the ANOVA results to another new sheet
    anova_df.to_excel(writer, sheet_name='ANOVA Results', index=True)